In [ ]:
!pip install torch transformers peft datasets bitsandbytes
!pip install accelerate>=0.26.0

In [ ]:
from huggingface_hub import list_datasets
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes
#List out the data set available
class_dataset = list_datasets(full=True, filter="text-classification")
classification_dataset_names = [dataset.id for dataset in class_dataset]

print(f"There are {len(classification_dataset_names)} classification datasets available on the hub")
print(f"The first 10 are: {classification_dataset_names[:10]}")

In [3]:
tweet_sent = load_dataset("ckandemir/bitcoin_tweets_sentiment_kaggle")

In [ ]:
tweet_sent

In [5]:
train_ds = tweet_sent["train"]

In [ ]:
train_ds

In [ ]:
train_ds[0]

In [ ]:
train_ds.column_names

In [ ]:
print(train_ds.features)

In [ ]:
print(train_ds['text'][:5])

In [11]:
import pandas as pd
tweet_sent.set_format(type="pandas")
df = tweet_sent["train"][:]

In [ ]:
df.head(10)

In [13]:
#Remove stop words
#Remove @ and url

In [14]:
tweet_sent_df = df.drop(columns=['Date'])

In [ ]:
tweet_sent_df.head(10)

In [ ]:
tweet_sent_df["Sentiment"].unique()

In [ ]:
!pip install -U scikit-learn

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
from sklearn.preprocessing import LabelEncoder

# Convert sentiment labels to integers
def convert_str_int_col(df, column_name, custom_map=None):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")
    if df[column_name].isnull().any():
        raise ValueError(f"Column '{column_name}' contains missing values.")
    if custom_map:
        unique_values = df[column_name].unique()
        for value in unique_values:
            if value not in custom_map:
                raise ValueError(f"Value '{value}' in column '{column_name}' is not present in the custom_map.")
        df[f"{column_name}_int"] = df[column_name].map(custom_map)
        mapping = custom_map
    else:
        label_encoder = LabelEncoder()
        df[f"{column_name}_int"] = label_encoder.fit_transform(df[column_name])
        mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    return df, mapping

custom_map = {"Positive": 1, "Neutral": 0, "Negative": 2}
tweet_sent_df, mapping = convert_str_int_col(tweet_sent_df, "Sentiment", custom_map)

In [ ]:
tweet_sent_df.head(10)

In [ ]:
!pip install emoji nltk

In [ ]:
import re
import emoji
from nltk.corpus import stopwords
import torch.nn as nn
import nltk

# Ensure NLTK stopwords are downloaded
nltk.download("stopwords")

class FeatureCleaner(nn.Module):
    """
    A feature cleaning module for text preprocessing.
    Performs operations like URL removal, hashtag removal, stopword removal, etc.
    """

    def __init__(self, slang_dict=None):
        """
        Initialize the FeatureCleaner.

        Args:
            slang_dict (dict, optional): A dictionary of slang terms and their expansions.
                                         Defaults to a predefined set of slang terms.
        """
        super().__init__()
        # Load stopwords
        self.stop_words = set(stopwords.words("english"))
        # Define slang dictionary (customizable)
        self.slang_dict = slang_dict or {
            "HODL": "hold on for dear life",
            "FOMO": "fear of missing out",
        }
        # Compile regex patterns for efficiency
        self.url_pattern = re.compile(r"http\S+|www\S+|https\S+", re.MULTILINE)
        self.hashtag_pattern = re.compile(r"@\w+|#\w+")
        self.special_char_pattern = re.compile(r"[^a-zA-Z0-9\s]")
        self.date_pattern = re.compile(
            r"\b\d{4}-\d{2}-\d{2}\b|\b\d{2}/\d{2}/\d{4}\b|\b\w{3,9}\s\d{1,2},?\s\d{4}\b"
        )
        # Compile slang replacement pattern
        self.slang_pattern = re.compile("|".join(re.escape(key) for key in self.slang_dict.keys()))

    def remove_url(self, text):
        """
        Remove URLs from the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with URLs removed.
        """
        if not text:
            return text
        return self.url_pattern.sub("", text)

    def remove_hashtags(self, text):
        """
        Remove hashtags and mentions from the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with hashtags and mentions removed.
        """
        if not text:
            return text
        return self.hashtag_pattern.sub("", text)

    def remove_special_characters(self, text):
        """
        Remove special characters from the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with special characters removed.
        """
        if not text:
            return text
        return self.special_char_pattern.sub("", text)

    def to_lowercase(self, text):
        """
        Convert text to lowercase.

        Args:
            text (str): Input text.

        Returns:
            str: Lowercase text.
        """
        if not text:
            return text
        return text.lower()

    def demoji(self, text):
        """
        Convert emojis to their text representations.

        Args:
            text (str): Input text.

        Returns:
            str: Text with emojis converted to text.
        """
        if not text:
            return text
        return emoji.demojize(text)

    def remove_stop_words(self, text):
        """
        Remove stopwords from the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with stopwords removed.
        """
        if not text:
            return text
        return " ".join([word for word in text.split() if word not in self.stop_words])

    def expand_slangs(self, text):
        """
        Expand slang terms in the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with slang terms expanded.
        """
        if not text:
            return text
        return self.slang_pattern.sub(lambda x: self.slang_dict[x.group()], text)

    def remove_dates(self, text):
        """
        Remove dates from the text.

        Args:
            text (str): Input text.

        Returns:
            str: Text with dates removed.
        """
        if not text:
            return text
        return self.date_pattern.sub("", text)

    def forward(self, text, remove_stopwords=True):
        """
        Apply all cleaning operations to the text.

        Args:
            text (str): Input text.
            remove_stopwords (bool, optional): Whether to remove stopwords. Defaults to True.

        Returns:
            str: Cleaned text.
        """
        if not text:
            return text

        # Apply cleaning operations in sequence
        text = self.remove_url(text)
        text = self.remove_hashtags(text)
        text = self.remove_special_characters(text)
        text = self.to_lowercase(text)
        text = self.demoji(text)
        text = self.expand_slangs(text)
        text = self.remove_dates(text)

        if remove_stopwords:
            text = self.remove_stop_words(text)

        return text

In [23]:
feature_cleaner = FeatureCleaner()

In [24]:
tweet_sent_df['text'] = tweet_sent_df['text'].apply(lambda x: feature_cleaner.forward(x))

In [ ]:
tweet_sent_df.head(10)

In [ ]:
tweet_sent_df.loc[3,"text"]

In [ ]:
# Count the number of empty or NaN rows in the 'text' column
empty_row_count = tweet_sent_df['text'].isna().sum() + (tweet_sent_df['text'].str.strip() == "").sum()
print(f"Number of empty rows: {empty_row_count}")


In [28]:
# Drop empty rows
tweet_sent_df = tweet_sent_df.dropna(subset=["text", "Sentiment", "Sentiment_int"])
tweet_sent_df = tweet_sent_df[~(tweet_sent_df["text"].str.strip() == "")]


In [ ]:
tweet_sent_df['text'].isna().sum()

In [ ]:
tweet_sent_df.head(10)

In [31]:
# Prepare test and validation datasets
df_test = tweet_sent["test"][:]
df_val = tweet_sent["eval"][:]

tweet_sent_test = df_test.drop(columns=['Date'])
tweet_sent_val = df_val.drop(columns=['Date'])

tweet_sent_test, _ = convert_str_int_col(tweet_sent_test, "Sentiment", custom_map)
tweet_sent_val, _ = convert_str_int_col(tweet_sent_val, "Sentiment", custom_map)

In [45]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

In [46]:
# Enable CUDA debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# Verify data types in the DataFrame
print("Data types in training DataFrame:")
print(tweet_sent_df.dtypes)

In [48]:
# Convert DataFrames to datasets with explicit type conversion
train_data = {
    'text': tweet_sent_df['text'].tolist(),
    'labels': tweet_sent_df['Sentiment_int'].astype(int).tolist()  # Ensure integer labels
}

val_data = {
    'text': tweet_sent_val['text'].tolist(),
    'labels': tweet_sent_val['Sentiment_int'].astype(int).tolist()  # Ensure integer labels
}

In [49]:
# Create datasets
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

In [50]:
# Load tokenizer
llama_model = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(llama_model)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = "right"

In [ ]:
# Tokenization function
def tokenize_function(examples):
    # Ensure text entries are strings
    texts = [str(text) for text in examples['text']]
    
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors=None
    )

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Configure model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32
)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    llama_model,
    num_labels=3,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
# Update model configuration
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

In [54]:
# Apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj']
)

model = get_peft_model(model, lora_config)

In [55]:
# Prepare datasets for training
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=8,
    optim="adamw_torch",
    no_cuda=False,
    fp16=False,
    bf16=False
)

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()